In [ ]:
!pip install selenium
!pip install bs4

In [ ]:
import requests
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver

In [ ]:
# selenium을 백그라운드로 실행하기 위해 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')

In [ ]:
# 우리가 데이터를 가져올 웹 페이지입니다.
Canna_opgg_url = 'https://www.op.gg/summoner/userName=T1%20Canna'

# selenium이 제어할 chrome을 실행합니다.
driver = webdriver.Chrome(r"./chromedriver")
# driver = webdriver.Chrome("/Users/namshin/chromedriver", options= options)

<ipython-input-108-da330690e2c3>:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r"./chromedriver")


In [ ]:
# 페이지로 이동.
driver.get(Canna_opgg_url)

In [ ]:
# HTML 정보 뽑기
Canna_total_html = driver.page_source

In [ ]:
# 파싱
Canna_total_soup = BeautifulSoup(Canna_total_html, 'html.parser')

In [ ]:
# list 생성
Canna_champions = []
Canna_kills = []
Canna_deaths = []
Canna_assists = []
Canna_results = []
Canna_Level = []

In [ ]:
# 전체 html 코드 중 우리가 원하는 selector를 만족하는 것만 가져오기
Canna_total_games_html = Canna_total_soup.select('div.GameItemList div.GameItemWrap')

In [ ]:
# 로딩된 모든 게임 수 출력
total_game_len = len(Canna_total_games_html)
print(total_game_len)

20


In [ ]:
# 각 게임에 대해 웹 페이지에 기재된 스탯을 찾아서(selector 사용) 결과 리스트에 append하기
for i in range(total_game_len):
    Canna_champions.append(list(Canna_total_games_html[i].select_one('div.ChampionName').
                                               stripped_strings)[0])
    Canna_kills.append(list(Canna_total_games_html[i].select_one('div.KDA div.KDA span.Kill').
                                               stripped_strings)[0])
    Canna_deaths.append(list(Canna_total_games_html[i].select_one('div.KDA div.KDA span.Death').
                                               stripped_strings)[0])
    Canna_assists.append(list(Canna_total_games_html[i].select_one('div.KDA div.KDA span.Assist').
                                               stripped_strings)[0])
    Canna_results.append(list(Canna_total_games_html[i].select_one('div.GameStats div.GameResult').
                                               stripped_strings)[0])
    Canna_Level.append(list(Canna_total_games_html[i].select_one('div.Stats div.Level ').
                                               stripped_strings)[0])

In [ ]:
# selenium이 제어하는 크롬 종료
driver.quit()

In [ ]:
Canna_kills = [int (i) for i in Canna_kills]
Canna_deaths = [int (i) for i in Canna_deaths]
Canna_assists = [int (i) for i in Canna_assists]

Canna_KDA = []
for i in range(total_game_len):
    if(Canna_deaths[i]==0):
        Canna_KDA.append("Perfect")
    else:
        Canna_KDA.append((Canna_kills[i]+Canna_assists[i])/Canna_deaths[i])

In [ ]:
# DataFrame으로 변환 후 출력
Canna_total_df = pd.DataFrame([Canna_champions,
                                Canna_results,
                                Canna_kills,
                                Canna_deaths,
                                Canna_assists,
                                Canna_KDA,
                                Canna_Level],
                               index = ['champion', 'result', 'kills', 'deaths', 'assists', 'KDA', 'Level']).T
Canna_total_df

,champion,result,kills,deaths,assists,KDA,Level
0,요네,승리,3,7,3,0.857143,레벨13
1,그웬,패배,1,9,4,0.555556,레벨13
2,리븐,승리,7,8,4,1.375,레벨16
3,리븐,패배,1,3,1,0.666667,레벨11
4,사일러스,패배,5,6,4,1.5,레벨16
5,니달리,승리,11,4,9,5,레벨12
6,트린다미어,승리,7,1,4,11,레벨14
7,리븐,승리,5,3,12,5.66667,레벨16
8,아트록스,승리,2,3,1,1,레벨11
9,사일러스,패배,14,7,2,2.28571,레벨16
